# Project Pipeline

## 0. Import

In [32]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
import json

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import linear_model, metrics

## 1. Tách dữ liệu để huấn luyện và kiểm thử
Mục đích của bước này là chia dữ liệu làm 2 phần, một phần để huấn luyện và phần còn lại coi như là thực tế và để đánh giá cuối cùng.
- Đầu vào: đường dẫn đến file dữ liệu đã crawl
- Đầu ra: 1 file train.csv và 1 file test.csv

In [2]:
def split_train_test(data_path, train_data_path, test_data_path):
    data_df = pd.read_csv(data_path, index_col=0, sep='\t')
    data_df.label.astype(int)
    
    train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=0)
    train_df.to_csv(train_data_path)
    test_df.to_csv(test_data_path)
    
split_train_test('../data/csv/vietnamnews.csv', '../data/csv/train.csv', '../data/csv/test.csv')

In [3]:
data_df = pd.read_csv('../data/csv/train.csv', index_col = 0)

y_sr = data_df["label"]
X_df = data_df.drop("label", axis=1)

## 2. Thống kê dữ liệu tập huấn luyện
- Kiểm tra các giá trị thiếu
- Kiểm tra độ dài trung bình của nội dung và tiêu đề của các thể loại, và vẽ biểu đồ

In [4]:
def get_data_insight(X_df):
    raise NotImplementedError

myset = set()
for index, row in X_df.iterrows():
    text = row['text']
    title = row['title']
    prev_len = len(myset)
    myset.add(text)
    if len(myset) == prev_len:
        print(text)
        
X_df.describe()

# from requests_html import HTMLSession
# sess = HTMLSession()

# r = sess.get('https://vietnamnews.vn/politics-laws/570613/trial-of-da-nangs-former-top-leaders-opens.html')
# findable_html = r.html
# findable_html.find('.vnnews-text-post', first=True).text

'Chairman of the State Duma of the Federal Assembly of the Russian Federation Vyacheslav Viktorovich Volodin. — VNA/VNS Photo'


title  text
count                                                4299  4299
unique                                               4299  4296
top     'VN, Nigeria to increase two-way commerce, inv...    ''
freq                                                    1     4                                                   title  \
count                                               4436   
unique                                              4436   
top     'VN team wins ASEAN Data Science Explorers 2019'   
freq                                                   1   

                                                     text  
count                                                4436  
unique                                               4435  
top     'Chairman of the State Duma of the Federal Ass...  
freq                                                    2

In [5]:
# TEST CELL

## 3. Tiền xử lý dữ liệu

In [5]:
# Module được thực hiện đầu Pipeline, xóa các khoảng trắng liên tiếp, các ký tự: \n, \t, ...

class RemoveIrrelevant(BaseEstimator, TransformerMixin):
    @staticmethod
    def clean_str(string):
        string = re.sub(r"\\xa0", " ", string)
        string = re.sub(r"\'s", "", string)
        string = re.sub(r"\'ve", "", string)
        string = re.sub(r"n\'t", "", string)
        string = re.sub(r"\'re", "", string)
        string = re.sub(r"\'d", "", string)
        string = re.sub(r"\'ll", "", string)
        string = re.sub(r"'", "", string)
        string = re.sub(r"\\n", " ", string)
        string = re.sub(r"\\t", " ", string)
        string = re.sub(r"\s{2,}", " ", string) # Consecutive spaces
        return string.strip()
    
    def fit(self, X_df, y=None):
        return self
    
    def transform(self, X_df, y=None):
        transform_X_df = X_df.copy()
        transform_X_df['title'] = transform_X_df['title'].apply(lambda x: self.clean_str(x))
        transform_X_df['text'] = transform_X_df['text'].apply(lambda x: self.clean_str(x))
        return transform_X_df

In [6]:
# TEST CELL
pl = RemoveIrrelevant()
transform_X = pl.transform(X_df)
print(X_df['text'][5])
print('\n')
print(transform_X['text'][5])
len(transform_X['text'][5])

'National Assembly (NA) Chairwoman Nguyễn Thị Kim Ngân (L) arrives\xa0at Minsk International Airport on Thursday morning (local time). — VNA/VNS Photo Trọng Đức\nMINSK —\xa0National Assembly (NA) Chairwoman Nguyễn Thị Kim Ngân and her entourage arrived at Minsk International Airport yesterday morning, beginning a three-day official visit to Belarus.\nThe trip is made at the invitation of Chairman of the Council of the Republic Mikhail Myasnikovich and Chairman of the House of Representatives of the NA of\xa0Belarus Vladimir Andreichenko.\n\nWelcoming the Vietnamese delegation at the airport were Deputy Chairman of the Council of the Republic Isachenko Anatoly Mikhailovich, other officials of the Council, and Vietnamese Ambassador to Belarus Phạm Hải.\n\nThe first official trip to Belarus by Chairwoman Ngân aims to enhance the traditional friendship and multifaceted cooperation between the two countries and push ahead with the implementation of cooperation agreements between the two par

1726

### 3.1.  Chọn các đặc trưng
+ Số ký tự văn bản
+ Số từ trong văn bản
+ Số lượng dấu cảm thán (!, ?)
+ Số chữ cái viết hoa
+ Số chữ cái viết thường
+ Các giá trị TF-IDF cho các từ trong văn bản (có xử lý văn bản trước đó), có những ưu tiên cho các từ trong title

#### 3.1.1 Các đặc trưng cơ bản

In [50]:
class NumTextCharFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    
    def transform(self, X_df, y=None):
        return X_df['text'].apply(lambda x: len(x)).values
    
class NumTextTokenFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    
    def transform(self, X_df, y=None):
        return X_df['text'].apply(lambda x: len(x.split(' '))).values
    
    
class NumTextExclamationMarkFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    
    def transform(self, X_df, y=None):
        return X_df['text'].apply(lambda x: (x.count('!')+x.count('?'))).values
    
class NumTextLowerCaseFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    
    def transform(self, X_df, y=None):
        return X_df['text'].str.count(r'[a-z]').values

class NumTextUpperCaseFeature(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    
    def transform(self, X_df, y=None):
        return X_df['text'].str.count(r'[A-Z]').values


In [51]:
# TEST CELL
pl = NumTextCharFeature()
transform_X = pl.transform(X_df)
print(transform_X)

pl = NumTextExclamationMarkFeature()
transform_X = pl.transform(X_df)
print(transform_X)

[2178 1902 1351 ... 1189 1469 3680]
[0 0 0 ... 0 0 0]


#### 3.1.2 Thử nghiệm TF-IDF trên 2 cột title và text và đưa ra vài heuristic về sự ưu tiên cho cột title

In [8]:
# Các module tiền xử lý trước khi tính tf-idf

class RemoveTone(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass
    
class LowerCase(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        pass
    
    def transform(self, X_df, y=None):
        pass
        

In [9]:
# TF-IDF

### 3.2.  Pipeline cho quá trình tiền xử lý

In [29]:
union = FeatureUnion([('num_text_char_feature', NumTextCharFeature()),
                      ('num_text_token_feature', NumTextTokenFeature()),
                      ('num_text_exclamation_mark_feature', NumTextExclamationMarkFeature()),
                      ('num_text_lower_case_feature', NumTextLowerCaseFeature()),
                      ('num_text_upper_case_feature', NumTextUpperCaseFeature())]) # !!!add tf-idf feature

preprocess_pipeline = Pipeline(steps=[('remove_irrelevant', RemoveIrrelevant()),
                                      ('union', union)])

## 4. Thiết kế mô hình hoàn chỉnh

### 4.1. Chia k fold 

In [34]:
def split_k_folds(k):
    return KFold(n_splits=k)

### 4.2. Khai báo các classifier

In [48]:
clf_list = []

# classifier parameters config
hyper_params_cfg = {
    'mlp':{
        'hidden_layer_sizes': [],
        'activation': [],
        'solver': []
    },
    'svm':{
        'param1': [],
        'param2': []
    }
}
model = linear_model.LogisticRegression(solver="lbfgs",multi_class="auto",max_iter=4000)

### 4.3. Huấn luyện

In [12]:
# Khi train mới fit_transform và transform tại mỗi vòng for của fold

In [52]:
k_fold = split_k_folds(k=5)
scores = []

for train_index, valid_index in k_fold.split(X_df):
    X_train, X_valid = X_df.iloc[train_index], X_df.iloc[valid_index]
    y_train, y_valid = y_sr.iloc[train_index], y_sr.iloc[valid_index]
#     print(X_train[:10])
#     print('\n')
#     print(y_train[:10])
    
    full_pipeline = Pipeline(steps=[('preprocess_pipeline', preprocess_pipeline), 
                                    ('classifier', model)])
    
    full_pipeline.fit_transform(X_train, y_train)
    y_valid_predict = full_pipeline.predict(X_val)
    
    score = metrics.accuracy_score(y_valid_predict, y_valid)
    scores.append(score)
    print(score)

ValueError: Expected 2D array, got 1D array instead:
array=[3036. 1064. 2662. ...   61.  141.  134.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## 5. Kết quả

In [53]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=20)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=10)
    plt.yticks(tick_marks, classes, fontsize=10)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=18)
    plt.xlabel('Predicted label', fontsize=18)

In [ ]:
cnf_matrix = confusion_matrix(y_test_1d, y_pred_1d)
plt.figure(figsize=(12,8))
plot_confusion_matrix(cnf_matrix, classes=text_labels, title="Confusion matrix",cmap="Greens")
plt.show()